In [1]:
import tkinter as tk
from tkinter import filedialog, Label, Button, Frame, messagebox
from PIL import Image, ImageTk  # Pillow for image handling in Tkinter
import numpy as np
import cv2
import os

In [2]:
try:
    from tensorflow.keras.models import load_model
    from tensorflow.keras.utils import to_categorical # Not strictly needed for prediction here but in your script
except ImportError:
    from keras.models import load_model
    from keras.utils import to_categorical

In [3]:
# --- Configuration (derived from your script) ---
MODEL_DIR = "model"
# Prioritize .keras, fallback to .hdf5 if user hasn't updated their script
MODEL_WEIGHTS_PATH_KERAS = os.path.join(MODEL_DIR, "cnn_weights.keras")
MODEL_WEIGHTS_PATH_HDF5 = os.path.join(MODEL_DIR, "cnn_weights.hdf5")

# Image processing parameters
IMG_WIDTH, IMG_HEIGHT = 32, 32
# Class labels from your script
CLASS_LABELS = ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']

# --- Global Variables for GUI State ---
loaded_model = None
selected_image_path = None
tk_image_display = None  # For displaying the image in Tkinter


In [4]:
def build_and_load_model():
    """
    Builds the CNN architecture (matching the one used for saving weights)
    and loads the saved weights.
    """
    global loaded_model
    # Define the same CNN architecture as used in your training script
    # This is crucial if you only saved weights. If you saved the full model,
    # load_model is simpler. The provided script uses load_weights OR trains.
    # For deployment, we assume weights are saved.

    # Let's try loading the full model first, as ModelCheckpoint usually saves that
    # when save_best_only=True and save_weights_only=False (default).
    model_path_to_try = None
    if os.path.exists(MODEL_WEIGHTS_PATH_KERAS):
        model_path_to_try = MODEL_WEIGHTS_PATH_KERAS
    elif os.path.exists(MODEL_WEIGHTS_PATH_HDF5):
        model_path_to_try = MODEL_WEIGHTS_PATH_HDF5

    if model_path_to_try:
        try:
            # Keras's ModelCheckpoint saves the entire model by default when using .keras or .h5
            # (unless save_weights_only=True was set, which wasn't in your snippet for cnn_model).
            loaded_model = load_model(model_path_to_try)
            print(f"Model loaded successfully from {model_path_to_try}")
            # loaded_model.summary() # Optional: print summary to console
            return True
        except Exception as e:
            messagebox.showerror("Model Load Error", f"Error loading model from {model_path_to_try}:\n{e}\nPlease ensure the model architecture matches or the file is a full model save.")
            print(f"Error loading model: {e}")
            return False
    else:
        messagebox.showerror("Model Load Error", f"Model weights file not found at:\n{MODEL_WEIGHTS_PATH_KERAS}\nor\n{MODEL_WEIGHTS_PATH_HDF5}\nPlease train and save the model first using your training script.")
        return False

In [5]:
def preprocess_image_for_prediction(image_path):
    """
    Loads and preprocesses a single image for CNN prediction.
    Matches the preprocessing in your script's `predict` function.
    """
    try:
        img_cv = cv2.imread(image_path)
        if img_cv is None:
            print(f"Error: Could not read image at {image_path}")
            return None
        
        img_resized = cv2.resize(img_cv, (IMG_WIDTH, IMG_HEIGHT))
        
        # Model expects 3 channels (as per your CNN input_shape if from original X_train)
        # If your images are grayscale, ensure they are converted to 3 channels if model expects it,
        # or that the model is trained on grayscale. Your training script implies X is color.
        if len(img_resized.shape) == 2: # Grayscale
            img_resized = cv2.cvtColor(img_resized, cv2.COLOR_GRAY2BGR)
        elif img_resized.shape[2] == 1: # Grayscale with 1 channel
             img_resized = cv2.cvtColor(img_resized, cv2.COLOR_GRAY2BGR)
        
        img_array = np.array(img_resized)
        img_array = img_array.astype('float32') / 255.0  # Normalize
        img_array = img_array.reshape(1, IMG_HEIGHT, IMG_WIDTH, 3)  # Reshape for model (1 sample, height, width, channels)
        return img_array
    except Exception as e:
        print(f"Error preprocessing image {image_path}: {e}")
        messagebox.showerror("Image Error", f"Could not process image: {e}")
        return None


In [6]:
# --- Tkinter GUI Functions ---

def select_image_button_action():
    """
    Handles the 'Select Image' button click.
    Opens a file dialog, loads and displays the image.
    """
    global selected_image_path, tk_image_display, original_image_for_display
    
    # Clear previous results and image
    result_label.config(text="Prediction: ")
    if image_display_label:
        image_display_label.config(image='') # Clear previous image

    file_path = filedialog.askopenfilename(
        title="Select Brain MRI Image",
        filetypes=(("Image files", "*.jpg *.jpeg *.png *.bmp *.tif"), ("All files", "*.*"))
    )
    if file_path:
        selected_image_path = file_path
        try:
            # Load image for display (can be larger than model input size)
            img_pil = Image.open(selected_image_path)
            original_image_for_display = img_pil.copy() # Keep a copy for re-display
            
            # Resize for display in the GUI
            display_width = 300
            w_percent = (display_width / float(img_pil.size[0]))
            h_size = int((float(img_pil.size[1]) * float(w_percent)))
            img_pil_resized = img_pil.resize((display_width, h_size), Image.LANCZOS) # Use Image.LANCZOS for Pillow >= 9.1.0 or Image.ANTIALIAS for older

            tk_image_display = ImageTk.PhotoImage(img_pil_resized)
            image_display_label.config(image=tk_image_display)
            image_display_label.image = tk_image_display # Keep a reference
            
            path_display_label.config(text=f"Selected: ...{os.path.basename(selected_image_path)}", wraplength=380)

        except Exception as e:
            messagebox.showerror("Image Load Error", f"Failed to load or display image: {e}")
            selected_image_path = None
            path_display_label.config(text="Selected: None")


In [7]:
def predict_button_action():
    """
    Handles the 'Predict Tumor Type' button click.
    """
    global loaded_model, selected_image_path
    
    if loaded_model is None:
        messagebox.showerror("Error", "Model not loaded. Please restart the application or check logs.")
        return
    if selected_image_path is None:
        messagebox.showinfo("No Image", "Please select an image first.")
        return

    # Preprocess the image for the model
    processed_img_array = preprocess_image_for_prediction(selected_image_path)

    if processed_img_array is not None:
        try:
            # Make prediction
            prediction_probabilities = loaded_model.predict(processed_img_array)
            predicted_class_index = np.argmax(prediction_probabilities, axis=1)[0]
            
            if 0 <= predicted_class_index < len(CLASS_LABELS):
                predicted_label_str = CLASS_LABELS[predicted_class_index]
                result_text = f"Prediction: {predicted_label_str}"
            else:
                result_text = "Prediction: Unknown class index"
                print(f"Warning: Predicted class index {predicted_class_index} is out of bounds for labels.")

            result_label.config(text=result_text)

        except Exception as e:
            messagebox.showerror("Prediction Error", f"An error occurred during prediction: {e}")
            result_label.config(text="Prediction: Error")


In [9]:
# --- Main Application Setup ---
def main_gui():
    global image_display_label, result_label, path_display_label # Allow modification by functions

    # Create the main window
    root = tk.Tk()
    root.title("Brain Tumor Classification - Deployment")
    root.geometry("500x600") # Adjusted size for better layout
    root.configure(bg="#f0f0f0") # Light gray background

    # Attempt to load the model on startup
    if not build_and_load_model():
        # If model loading fails critically, you might want to disable prediction button or exit
        print("Critical error: Model could not be loaded. Prediction will not work.")
        # Consider adding a label to GUI indicating model load failure
    
    # --- GUI Elements ---
    title_label = Label(root, text="Brain Tumor Classifier", font=("Helvetica", 18, "bold"), bg="#f0f0f0", fg="#333")
    title_label.pack(pady=20)

    # Frame for buttons
    button_frame = Frame(root, bg="#f0f0f0")
    button_frame.pack(pady=10)

    select_button = Button(button_frame, text="Select Brain MRI Image", command=select_image_button_action, font=("Helvetica", 12), bg="#4CAF50", fg="white", relief="flat", padx=10, pady=5)
    select_button.pack(side=tk.LEFT, padx=10)

    predict_button = Button(button_frame, text="Predict Tumor Type", command=predict_button_action, font=("Helvetica", 12), bg="#007BFF", fg="white", relief="flat", padx=10, pady=5)
    predict_button.pack(side=tk.LEFT, padx=10)
    if loaded_model is None: # Disable button if model did not load
        predict_button.config(state=tk.DISABLED)


    # Label to display selected image path (truncated)
    path_display_label = Label(root, text="Selected: None", font=("Helvetica", 9), bg="#f0f0f0", fg="#555")
    path_display_label.pack(pady=(0,10))


    # Label to display the image (placeholder)
    image_display_label = Label(root, bg="#ddd", relief="sunken", width=42, height=18) # Approx 300px width for 32x32 upscaled
    image_display_label.pack(pady=10, padx=20, fill="both", expand=False) # Don't expand so it keeps its size roughly

    # Label to display the prediction result
    result_label = Label(root, text="Prediction: ", font=("Helvetica", 14, "bold"), bg="#f0f0f0", fg="#333")
    result_label.pack(pady=20)

    # Start the Tkinter event loop
    root.mainloop()

if __name__ == "__main__":
    # Ensure the 'model' directory exists (or the script won't find weights)
    if not os.path.isdir(MODEL_DIR):
        messagebox.showerror("Setup Error", f"The directory '{MODEL_DIR}' was not found. Please ensure it exists and contains the model weights.")
        # Optionally, exit if the model directory is critical
        # exit()
    main_gui()
import tkinter as tk
from tkinter import filedialog, Label, Button, Frame, messagebox
from PIL import Image, ImageTk  # Pillow for image handling in Tkinter
import numpy as np
import cv2
import os
# Use the modern Keras import if available, otherwise fallback for older TensorFlow/Keras
try:
    from tensorflow.keras.models import load_model
    from tensorflow.keras.utils import to_categorical # Not strictly needed for prediction here but in your script
except ImportError:
    from keras.models import load_model
    from keras.utils import to_categorical


# --- Configuration (derived from your script) ---
MODEL_DIR = "model"
# Prioritize .keras, fallback to .hdf5 if user hasn't updated their script
MODEL_WEIGHTS_PATH_KERAS = os.path.join(MODEL_DIR, "cnn_weights.keras")
MODEL_WEIGHTS_PATH_HDF5 = os.path.join(MODEL_DIR, "cnn_weights.hdf5")

# Image processing parameters
IMG_WIDTH, IMG_HEIGHT = 32, 32
# Class labels from your script
CLASS_LABELS = ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']

# --- Global Variables for GUI State ---
loaded_model = None
selected_image_path = None
tk_image_display = None  # For displaying the image in Tkinter


Model loaded successfully from model\cnn_weights.keras


In [10]:
# --- Core Logic for Prediction (adapted from your script) ---

def build_and_load_model():
    """
    Builds the CNN architecture (matching the one used for saving weights)
    and loads the saved weights.
    """
    global loaded_model
    # Define the same CNN architecture as used in your training script
    # This is crucial if you only saved weights. If you saved the full model,
    # load_model is simpler. The provided script uses load_weights OR trains.
    # For deployment, we assume weights are saved.

    # Let's try loading the full model first, as ModelCheckpoint usually saves that
    # when save_best_only=True and save_weights_only=False (default).
    model_path_to_try = None
    if os.path.exists(MODEL_WEIGHTS_PATH_KERAS):
        model_path_to_try = MODEL_WEIGHTS_PATH_KERAS
    elif os.path.exists(MODEL_WEIGHTS_PATH_HDF5):
        model_path_to_try = MODEL_WEIGHTS_PATH_HDF5

    if model_path_to_try:
        try:
            # Keras's ModelCheckpoint saves the entire model by default when using .keras or .h5
            # (unless save_weights_only=True was set, which wasn't in your snippet for cnn_model).
            loaded_model = load_model(model_path_to_try)
            print(f"Model loaded successfully from {model_path_to_try}")
            # loaded_model.summary() # Optional: print summary to console
            return True
        except Exception as e:
            messagebox.showerror("Model Load Error", f"Error loading model from {model_path_to_try}:\n{e}\nPlease ensure the model architecture matches or the file is a full model save.")
            print(f"Error loading model: {e}")
            return False
    else:
        messagebox.showerror("Model Load Error", f"Model weights file not found at:\n{MODEL_WEIGHTS_PATH_KERAS}\nor\n{MODEL_WEIGHTS_PATH_HDF5}\nPlease train and save the model first using your training script.")
        return False


In [11]:
def preprocess_image_for_prediction(image_path):
    """
    Loads and preprocesses a single image for CNN prediction.
    Matches the preprocessing in your script's `predict` function.
    """
    try:
        img_cv = cv2.imread(image_path)
        if img_cv is None:
            print(f"Error: Could not read image at {image_path}")
            return None
        
        img_resized = cv2.resize(img_cv, (IMG_WIDTH, IMG_HEIGHT))
        
        # Model expects 3 channels (as per your CNN input_shape if from original X_train)
        # If your images are grayscale, ensure they are converted to 3 channels if model expects it,
        # or that the model is trained on grayscale. Your training script implies X is color.
        if len(img_resized.shape) == 2: # Grayscale
            img_resized = cv2.cvtColor(img_resized, cv2.COLOR_GRAY2BGR)
        elif img_resized.shape[2] == 1: # Grayscale with 1 channel
             img_resized = cv2.cvtColor(img_resized, cv2.COLOR_GRAY2BGR)
        
        img_array = np.array(img_resized)
        img_array = img_array.astype('float32') / 255.0  # Normalize
        img_array = img_array.reshape(1, IMG_HEIGHT, IMG_WIDTH, 3)  # Reshape for model (1 sample, height, width, channels)
        return img_array
    except Exception as e:
        print(f"Error preprocessing image {image_path}: {e}")
        messagebox.showerror("Image Error", f"Could not process image: {e}")
        return None

In [12]:
# --- Tkinter GUI Functions ---

def select_image_button_action():
    """
    Handles the 'Select Image' button click.
    Opens a file dialog, loads and displays the image.
    """
    global selected_image_path, tk_image_display, original_image_for_display
    
    # Clear previous results and image
    result_label.config(text="Prediction: ")
    if image_display_label:
        image_display_label.config(image='') # Clear previous image

    file_path = filedialog.askopenfilename(
        title="Select Brain MRI Image",
        filetypes=(("Image files", "*.jpg *.jpeg *.png *.bmp *.tif"), ("All files", "*.*"))
    )
    if file_path:
        selected_image_path = file_path
        try:
            # Load image for display (can be larger than model input size)
            img_pil = Image.open(selected_image_path)
            original_image_for_display = img_pil.copy() # Keep a copy for re-display
            
            # Resize for display in the GUI
            display_width = 300
            w_percent = (display_width / float(img_pil.size[0]))
            h_size = int((float(img_pil.size[1]) * float(w_percent)))
            img_pil_resized = img_pil.resize((display_width, h_size), Image.LANCZOS) # Use Image.LANCZOS for Pillow >= 9.1.0 or Image.ANTIALIAS for older

            tk_image_display = ImageTk.PhotoImage(img_pil_resized)
            image_display_label.config(image=tk_image_display)
            image_display_label.image = tk_image_display # Keep a reference
            
            path_display_label.config(text=f"Selected: ...{os.path.basename(selected_image_path)}", wraplength=380)

        except Exception as e:
            messagebox.showerror("Image Load Error", f"Failed to load or display image: {e}")
            selected_image_path = None
            path_display_label.config(text="Selected: None")


In [13]:
def predict_button_action():
    """
    Handles the 'Predict Tumor Type' button click.
    """
    global loaded_model, selected_image_path
    
    if loaded_model is None:
        messagebox.showerror("Error", "Model not loaded. Please restart the application or check logs.")
        return
    if selected_image_path is None:
        messagebox.showinfo("No Image", "Please select an image first.")
        return

    # Preprocess the image for the model
    processed_img_array = preprocess_image_for_prediction(selected_image_path)

    if processed_img_array is not None:
        try:
            # Make prediction
            prediction_probabilities = loaded_model.predict(processed_img_array)
            predicted_class_index = np.argmax(prediction_probabilities, axis=1)[0]
            
            if 0 <= predicted_class_index < len(CLASS_LABELS):
                predicted_label_str = CLASS_LABELS[predicted_class_index]
                result_text = f"Prediction: {predicted_label_str}"
            else:
                result_text = "Prediction: Unknown class index"
                print(f"Warning: Predicted class index {predicted_class_index} is out of bounds for labels.")

            result_label.config(text=result_text)

        except Exception as e:
            messagebox.showerror("Prediction Error", f"An error occurred during prediction: {e}")
            result_label.config(text="Prediction: Error")


In [14]:
# --- Main Application Setup ---
def main_gui():
    global image_display_label, result_label, path_display_label # Allow modification by functions

    # Create the main window
    root = tk.Tk()
    root.title("Brain Tumor Classification - Deployment")
    root.geometry("500x600") # Adjusted size for better layout
    root.configure(bg="#f0f0f0") # Light gray background

    # Attempt to load the model on startup
    if not build_and_load_model():
        # If model loading fails critically, you might want to disable prediction button or exit
        print("Critical error: Model could not be loaded. Prediction will not work.")
        # Consider adding a label to GUI indicating model load failure
    
    # --- GUI Elements ---
    title_label = Label(root, text="Brain Tumor Classifier", font=("Helvetica", 18, "bold"), bg="#f0f0f0", fg="#333")
    title_label.pack(pady=20)

    # Frame for buttons
    button_frame = Frame(root, bg="#f0f0f0")
    button_frame.pack(pady=10)

    select_button = Button(button_frame, text="Select Brain MRI Image", command=select_image_button_action, font=("Helvetica", 12), bg="#4CAF50", fg="white", relief="flat", padx=10, pady=5)
    select_button.pack(side=tk.LEFT, padx=10)

    predict_button = Button(button_frame, text="Predict Tumor Type", command=predict_button_action, font=("Helvetica", 12), bg="#007BFF", fg="white", relief="flat", padx=10, pady=5)
    predict_button.pack(side=tk.LEFT, padx=10)
    if loaded_model is None: # Disable button if model did not load
        predict_button.config(state=tk.DISABLED)


    # Label to display selected image path (truncated)
    path_display_label = Label(root, text="Selected: None", font=("Helvetica", 9), bg="#f0f0f0", fg="#555")
    path_display_label.pack(pady=(0,10))


    # Label to display the image (placeholder)
    image_display_label = Label(root, bg="#ddd", relief="sunken", width=42, height=18) # Approx 300px width for 32x32 upscaled
    image_display_label.pack(pady=10, padx=20, fill="both", expand=False) # Don't expand so it keeps its size roughly

    # Label to display the prediction result
    result_label = Label(root, text="Prediction: ", font=("Helvetica", 14, "bold"), bg="#f0f0f0", fg="#333")
    result_label.pack(pady=20)

    # Start the Tkinter event loop
    root.mainloop()

if __name__ == "__main__":
    # Ensure the 'model' directory exists (or the script won't find weights)
    if not os.path.isdir(MODEL_DIR):
        messagebox.showerror("Setup Error", f"The directory '{MODEL_DIR}' was not found. Please ensure it exists and contains the model weights.")
        # Optionally, exit if the model directory is critical
        # exit()
    main_gui()

Model loaded successfully from model\cnn_weights.keras


C:\Users\sahit\AppData\Local\Temp\ipykernel_13464\2374666106.py:30: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img_pil_resized = img_pil.resize((display_width, h_size), Image.LANCZOS) # Use Image.LANCZOS for Pillow >= 9.1.0 or Image.ANTIALIAS for older


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step
